In [31]:
import xarray as xr
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

unc = xr.open_dataset('/data/acker/WashU_GL_unc/V5GL04.HybridPM25E.Global.201601-201612.nc').sel(lat=slice(14, 68), lon=slice(-170., -52))
na = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2016001-2016364.nc')
gl = xr.open_dataset('/data/acker/WashU_V5_GL/V5GL04.HybridPM25.Global.201601-201612.nc').sel(lat=slice(14, 68), lon=slice(-170., -52))

In [32]:
gl

<xarray.Dataset> Size: 255MB
Dimensions:  (lon: 11800, lat: 5400)
Coordinates:
  * lon      (lon) float64 94kB -170.0 -170.0 -170.0 ... -52.03 -52.01 -52.01
  * lat      (lat) float64 43kB 14.01 14.02 14.02 14.03 ... 67.97 67.99 68.0
Data variables:
    GWRPM25  (lat, lon) float32 255MB ...
Attributes:
    Title:            Hybrid PM_2_._5 [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         C:\Users\Aaron\Documents\MATLAB\PM25_wMelanie\GlobalMon...
    Format:           NetCDF-4
    Delta_Lat:        0.009998321533203125
    Delta_Lon:        0.0099945068359375
    SpatialCoverage:  Global
    Start_Date:       20160101
    End_Date:         20161231

In [33]:
na

<xarray.Dataset> Size: 255MB
Dimensions:  (lon: 11800, lat: 5400)
Coordinates:
  * lon      (lon) float64 94kB -170.0 -170.0 -170.0 ... -52.02 -52.02 -52.01
  * lat      (lat) float64 43kB 14.01 14.02 14.03 14.04 ... 67.97 67.98 68.0
Data variables:
    GWRPM25  (lat, lon) float32 255MB ...
Attributes:
    Title:            Hybrid PM_2_._5 [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         ./RegionalGWR-NA-BiWeekly-Geo20230823-GM20230905-V20240...
    Format:           NetCDF-4
    Delta_Lat:        0.01
    Delta_Lon:        0.01
    SpatialCoverage:  xNorthAmerica
    Start_Date:       2016001
    End_Date:         2016364

In [34]:
u = unc.to_dataframe().reset_index()
n = na.to_dataframe().reset_index()
g = gl.to_dataframe().reset_index()

In [35]:
u['lat'] = u['lat'].round(3)
u['lon'] = u['lon'].round(3)

n['lat'] = n['lat'].round(3)
n['lon'] = n['lon'].round(3)

g['lat'] = g['lat'].round(3)
g['lon'] = g['lon'].round(3)

In [36]:
g

,lon,lat,GWRPM25
0,-169.995,14.005,NaN
1,-169.995,14.015,NaN
2,-169.995,14.025,NaN
3,-169.995,14.035,NaN
4,-169.995,14.045,NaN
...,...,...,...
63719995,-52.005,67.955,1.2
63719996,-52.005,67.965,1.2
63719997,-52.005,67.975,1.2
63719998,-52.005,67.985,1.2


In [37]:
# Merge and calculate percent uncertainty
merged = pd.merge(g, u, left_index=True, right_index=True, how='inner')
merged.reset_index(inplace=True)
print(merged)
merged['percent'] = (merged['GWRPM25SIGMA'] / merged['GWRPM25']) * 100
merged = merged.rename(columns={'lat_x':'lat','lon_x':'lon'})    
    # Filter and return
remove = merged[merged['percent'] >= 90][['lat', 'lon', 'GWRPM25']]
remove

             index    lon_x   lat_x  GWRPM25    lon_y   lat_y  GWRPM25SIGMA
0                0 -169.995  14.005      NaN -169.995  14.005           NaN
1                1 -169.995  14.015      NaN -169.995  14.015           NaN
2                2 -169.995  14.025      NaN -169.995  14.025           NaN
3                3 -169.995  14.035      NaN -169.995  14.035           NaN
4                4 -169.995  14.045      NaN -169.995  14.045           NaN
...            ...      ...     ...      ...      ...     ...           ...
63719995  63719995  -52.005  67.955      1.2  -52.005  67.955           1.2
63719996  63719996  -52.005  67.965      1.2  -52.005  67.965           1.2
63719997  63719997  -52.005  67.975      1.2  -52.005  67.975           1.2
63719998  63719998  -52.005  67.985      1.2  -52.005  67.985           1.2
63719999  63719999  -52.005  67.995      1.2  -52.005  67.995           1.2

[63720000 rows x 7 columns]


,lat,lon,GWRPM25
301910,63.105,-169.445,2.6
301911,63.115,-169.445,2.6
301912,63.125,-169.445,2.6
301913,63.135,-169.445,2.6
301914,63.145,-169.445,2.6
...,...,...,...
63719995,67.955,-52.005,1.2
63719996,67.965,-52.005,1.2
63719997,67.975,-52.005,1.2
63719998,67.985,-52.005,1.2


In [38]:
# Identify matching lat and lon from 'remove' in 'n'
filtered_n = n.merge(remove[['lat', 'lon']], on=['lat', 'lon'], how='left', indicator=True)

# Keep only rows that do not have a match in 'remove'
filtered_n = filtered_n[filtered_n['_merge'] == 'left_only']

# Drop the '_merge' column, as it is no longer needed
filtered_n = filtered_n.drop(columns=['_merge'])

# Print the resulting dataframe

filtered_n

,lon,lat,GWRPM25
0,-169.995,14.005,NaN
1,-169.995,14.015,NaN
2,-169.995,14.025,NaN
3,-169.995,14.035,NaN
4,-169.995,14.045,NaN
...,...,...,...
63719863,-52.005,66.635,NaN
63719868,-52.005,66.685,NaN
63719869,-52.005,66.695,NaN
63719898,-52.005,66.985,NaN


In [39]:
filtered_n.dropna()

,lon,lat,GWRPM25
3878,-169.995,52.785,3.6
3879,-169.995,52.795,3.5
3880,-169.995,52.805,3.5
3881,-169.995,52.815,3.6
3882,-169.995,52.825,3.6
...,...,...,...
63388550,-52.615,47.505,3.7
63388551,-52.615,47.515,3.7
63388552,-52.615,47.525,3.7
63393951,-52.605,47.515,3.7


In [41]:
filtered_n['GWRPM25'].max()

46.0